# Wrapper & Adapter Design Patterns

Design patterns in MolPy architecture.

## Patterns

- Wrapper: Adds functionality
- Adapter: Converts interfaces

## See Also

- [Recipe System](recipe_system.ipynb)